# Plot Starlet planes with PyWI-CTA

## Import required modules

This notebook requires PyWI-CTA for the I/O.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

import pywicta
from pywicta.io import geometry_converter
from pywicta.io.images import image_generator
from pywicta.io.images import plot_ctapipe_image
from pywicta.io.images import plot_hillas_parameters_on_axes

from pywicta.denoising import starlet
from pywicta.denoising.starlet import WaveletTransform
from pywicta.denoising import inverse_transform_sampling
from pywicta.denoising.inverse_transform_sampling import EmpiricalDistribution

print(pywicta.get_version())

## Get image

The next cells define the list of images to use in this notebook.

Images can be fetched from Fits files or from Simtel files.
Fits files are much lighter and much faster to process than Simtel files but they are specific to PyWI-CTA and thus you first have to generate them from Simtel files using [the following script](). Also, contrary to Simtel files, a Fits files contains only one "image" (i.e. an unique event viewed from one unique telescope).

In [ ]:
#CAM_ID = "ASTRICam"
#CAM_ID = "CHEC"
#CAM_ID = "DigiCam"
#CAM_ID = "FlashCam"
#CAM_ID = "NectarCam"
CAM_ID = "LSTCam"

IMG_PATH_LIST = ["~/gamma_20deg_0deg_run103___cta-prod3-lapalma3-2147m-LaPalma.simtel.gz_TEL001_EV1002301.fits"]

image_gen = image_generator(IMG_PATH_LIST,
                            cam_filter_list=[CAM_ID],
                            ctapipe_format=False)

img = next(image_gen)

In [ ]:
img.meta

## Plot

Images comming from FITS files are stored in 2D arrays (mainly because the Sparce2D wavelet transform expects images stored in 2D arrays).
But ctapipe uses a special 1D array representation for IACT images (defined in "geom" objects).
Thus conversion has to be done prior to use ctapipe functions on images comming from Fits files (using the "geometry_converter" function).

In [ ]:
geom1d = geometry_converter.get_geom1d(CAM_ID)

In [ ]:
img_input_1d = geometry_converter.image_2d_to_1d(img.input_image, CAM_ID)

disp = plot_ctapipe_image(img_input_1d,
                          geom1d,
                          title='Noised image',
                          norm='lin',
                          plot_axis=False)

## Clean the image using the Starlet transform

### Get empirical noise distribution

In [ ]:
use_noise_distribution = True

if use_noise_distribution:
    noise_cdf_file = inverse_transform_sampling.get_cdf_file_path(CAM_ID)  # pywicta.denoising.cdf.LSTCAM_CDF_FILE
    print(noise_cdf_file)
    noise_distribution = EmpiricalDistribution(noise_cdf_file)
else:
    noise_distribution = None

### Setup the Starlet transform

Fell free to change the following default setup to get a better understanding of the Starlet transform...

In [ ]:
type_of_filtering = 'hard_filtering'
filter_thresholds = (3., 0.2)
clusters_threshold = 0.2
last_scale_treatment = 'drop'
detect_only_positive_structures = False
kill_isolated_pixels = True

### Compute the Starlet transform and plot the Starlet planes

In [ ]:
number_of_scales = len(filter_thresholds) + 1

in_planes = starlet.wavelet_transform(img.input_image,
                                      number_of_scales=number_of_scales,
                                      noise_distribution=noise_distribution)

filtered_in_planes = starlet.filter_planes(in_planes,
                                           method=type_of_filtering,
                                           thresholds=filter_thresholds,
                                           detect_only_positive_structures=detect_only_positive_structures)

for plane_index, (plane, filtered_plane) in enumerate(zip(in_planes, filtered_in_planes)):
    if plane_index < len(in_planes) - 1:
        significant_pixels_mask = (geometry_converter.image_2d_to_1d(filtered_plane, CAM_ID) != 0)
    else:
        significant_pixels_mask = None
        
    if plane_index == 0:
        title_comment = " (smallest scale)"
    elif plane_index == len(in_planes) - 2:
        title_comment = ' (largest scale)'
    elif plane_index == len(in_planes) - 1:
        title_comment = ' ("coarse" a.k.a. "residuals")'
    else:
        title_comment = ""
            
    disp = plot_ctapipe_image(geometry_converter.image_2d_to_1d(plane, CAM_ID),
                              geom1d,
                              title='Plane {}{}'.format(plane_index + 1, title_comment),
                              norm='lin',
                              highlight_mask=significant_pixels_mask,
                              plot_axis=False)